In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
#df.head() # จะโชว์แค่5อัน
df

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169688,47,2,0.0,0.0,0,0,1.0,0.0,5.0,4.0,...,42.0,5.0,2.0,1.1,0.0,0.0,0.0,0.0,0.0,2.0
3167041,39,1,0.0,0.0,0,0,1.0,0.0,5.0,2.0,...,20.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2623351,46,2,0.0,0.0,0,0,1.0,0.0,5.0,1.0,...,13.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_clean_column = df.drop(['size','appearance','Wardsize','RHlvsi','depth','nodeyiel','vgmargin','pelvicme','adnmet'],axis = 1)
df_clean_column.isnull().sum(axis=0)
# size กับ OPDsize หมอบอกคืออันเดียวกันเพราะงั้นลบซักอัน
# 0 row / 1 column

age            0
parity         0
hiv            4
menopaus       1
disease        0
surgery        0
conization     5
OPDsize       17
stage         24
pchemo         1
finalhisto    10
utmet         98
vgmet         97
pmmet         94
dtype: int64

In [4]:
df_clean = df_clean_column.dropna(axis = 0)
#แถวไหนมี missing data ดรอบไป

In [5]:
pel_class = np.unique(df_clean['pmmet'])
#pel_class = df_clean['pmmet'].unique()
pel_class

array([0., 1.])

In [6]:
pel_value = [sum(df_clean['pmmet']==pel_class[0]),
             sum(df_clean['pmmet']==pel_class[1])]
pel_value

[1291, 280]

In [7]:
# เราจะเอา ค่า pmmet มาเป็นคำตอบของ model ดังนั้นเลยเก็บค่าไว้ใ้ช้
y = df_clean['pmmet'].values # คำตอบ
X = df_clean.drop(['pmmet'],axis = 1).values # ข้อมูล

---------------

# HOLD OUT

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0) 

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=0) 

In [10]:
# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)

In [11]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_val_norm)
acc = sum(yp == y_val)/len(y_val)
print("HOLD OUT Trainging accuracy : "+str(acc))

HOLD OUT Trainging accuracy : 0.8174603174603174


****จะทำtest ให้เอาตัว training dataทั้งหมดมาสร้าง modelอีกรอบ แล้วเอาไปtest****

---------------------------------

# Cross Validation

|| Leave One Out

In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train)

In [14]:
from sklearn.model_selection import cross_val_score
clf = LogisticRegression(random_state=0, solver='liblinear')
acc = cross_val_score(clf,X_train_norm,y_train,cv=10) #ถ้าจะทำ--Leave One Out-- Cross Validation คือปรับ cv ให้เท่ากับขนาดข้อมูลทั้งหมด
print("10CV Training accuracy : "+str(acc))

10CV Training accuracy : [0.81746032 0.80952381 0.81746032 0.78571429 0.81746032 0.81746032
 0.8        0.832      0.816      0.824     ]


In [15]:
print("10CV Training accuracy : "+str(acc.mean()))

10CV Training accuracy : 0.8137079365079366


----------------------------

# Test accuracy

ของ data ใน *train ของ Hold out และ CV

In [16]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [17]:
# normali X_train_norm, X_test_norm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaler.fit(X_train) 
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [18]:
#model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm) # X_test_norm
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8761904761904762


-----------------
**HOLD OUT** Trainging accuracy : 0.817 <br>
**10CV** Training accuracy : 0.813  <br><br>

**Test accuracy** : 0.8761904761904762<br>


-------------------------


pre processing ที่ควรทำ (เพิ่มจากweekเก่า)

In [19]:
df.var(axis=0) # 0 คือ คอลลัม
# hiv  0.006936 แปลว่า ไม่ว่ามันจะอยู่กลุ่มไหนค่ามันก็จะเป็นแค่ค่าเดียว ค่าเดิม (เหมือนๆกันหมดทุกโรล) เอาไปเปรียบเทียบกับกลุ่มอื่นๆไมไ่ด้

age               81.065940
parity             1.795348
hiv                0.006936
menopaus           0.207860
disease            0.215740
surgery            0.234638
conization         0.226518
OPDsize            3.664245
appearance         2.997285
stage              1.383333
pchemo             0.149920
Wardsize           3.361144
finalhisto         0.744274
nodeyiel         114.208070
RHlvsi        231725.557678
depth              0.509143
size               3.207926
utmet              0.153174
vgmargin           0.189784
vgmet              0.357326
pelvicme           0.388821
pmmet              0.146025
adnmet             0.732903
dtype: float64

In [20]:
df['hiv'] # เท่ากับ0 เป็นค่าคงที่ เอาออกเลย ไม่มีประโยชน์ในการทำ

hn
2631840    0.0
2633481    0.0
2634477    0.0
2633633    0.0
2630496    0.0
          ... 
3169688    0.0
3167041    0.0
2623351    0.0
3302539    0.0
3855674    0.0
Name: hiv, Length: 1723, dtype: float64

In [21]:
df.shape # ดูว่ามีข้อมูลเท่่าไหร่

(1723, 23)

In [22]:
# ดรอบ โรลข้อมูลซ้่ำ
df_nodup = df.drop_duplicates()
df_nodup.shape # ได้เท่าเดิม แปลว่าไม่มีข้อมูลซ้ำ

(1723, 23)

In [23]:
# หา correlation ขเ้อมูลที่คล้ายกันมากๆ ยิ่งใกล้1ยิ่งคล้าย
s = df.corr().unstack()
so = s.sort_values(ascending=False)
so[0:50]
# OPDsize  Wardsize  0.875699 ยังถือว่า โอเค แต่ถ้า 0.97-0.98 เราต้องไปถามละว่ามันคืออันเดียวกันไหม มันซ้ำซ้อนกันรึเปล่า ถ้าซ้ำก็เอาออก

#  ถ้า training acc ูสูงเกิน 95% ให้คิดเลยว่าต้องมีอะไรผิด    
#  หลักๆเลยคือ ให้ไปดู correlation ระว่าง feature กะับ label ถ้าเข้าใกล้ 1 **แสดงว่า feature นั้นคือ label **แต่เราเอามาเป็น feature

age         age           1.000000
finalhisto  finalhisto    1.000000
menopaus    menopaus      1.000000
disease     disease       1.000000
surgery     surgery       1.000000
conization  conization    1.000000
OPDsize     OPDsize       1.000000
appearance  appearance    1.000000
stage       stage         1.000000
pchemo      pchemo        1.000000
nodeyiel    nodeyiel      1.000000
parity      parity        1.000000
RHlvsi      RHlvsi        1.000000
depth       depth         1.000000
size        size          1.000000
utmet       utmet         1.000000
vgmargin    vgmargin      1.000000
vgmet       vgmet         1.000000
pelvicme    pelvicme      1.000000
pmmet       pmmet         1.000000
hiv         hiv           1.000000
Wardsize    Wardsize      1.000000
adnmet      adnmet        1.000000
OPDsize     Wardsize      0.875699
Wardsize    OPDsize       0.875699
size        Wardsize      0.804733
Wardsize    size          0.804733
appearance  conization    0.794773
conization  appearan

In [24]:
so[20:30]

hiv         hiv           1.000000
Wardsize    Wardsize      1.000000
adnmet      adnmet        1.000000
OPDsize     Wardsize      0.875699
Wardsize    OPDsize       0.875699
size        Wardsize      0.804733
Wardsize    size          0.804733
appearance  conization    0.794773
conization  appearance    0.794773
age         menopaus      0.721046
dtype: float64

-----------------

# Feature Reduction

ไม่แนะนำให้ยุบ เพราะ เวลายุบ ความหมายจะ หาย ทำให้ตอบไม่ได้ว่าfeatureตัวไหนมีผลกับการหาค่า <br>
ทำแล้วโมเดลอาจแม่นขึ้น แต่ส่วนใหญ่แม่นน้อยลง

# PCA - Principle Component Analysis

In [25]:
X_train.shape

(1256, 13)

In [26]:
# ปกติมักทำก่อน normalization
from sklearn.decomposition import PCA
pca = PCA(n_components = 5) # ลดFeatureจาก13 ให้เหลือ5
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_pca)
X_train_norm = scaler.transform(X_train_pca)
X_test_norm = scaler.transform(X_test_pca)

In [27]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8507936507936508


# t-SNE 

In [28]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2)
tsne.fit(X_train)
X_train_tsne = tsne.fit_transform(X_train)
X_test_tsne = tsne.fit_transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_tsne)
X_train_norm = scaler.transform(X_train_tsne)
X_test_norm = scaler.transform(X_test_tsne)

In [29]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8507936507936508


-----------------------
# Feature Selection <br>สำคัญมาก

เลือก Feature ที่มีความสำคัญมากๆ มาใช้ 

# Filter Method
#  SelectKBest || Filter Method || ANOVA f-value

In [30]:
#feature_selection
from sklearn.feature_selection import SelectKBest # SelectKBest เลือก Feature ที่ดีที่สุดตามค่า ที่เราสนใจออกมา K ตัว
from sklearn.feature_selection import f_classif   # ANOVA f-value
fs = SelectKBest(f_classif, k=5)
fs.fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_fs)
X_train_norm = scaler.transform(X_train_fs)
X_test_norm = scaler.transform(X_test_fs)

In [31]:
# ดูว่าเลือก Feature อะไรมาบ้าง

np.argsort(-fs.scores_)

# ก็คือเอาคะแนนมากที่สุดมา 5 ตัว ดูโดยเรียงจากมากไปน้อย จะ ได้ Feature 7,  8, 11,  6, 12

array([ 7,  8, 11,  6, 12,  3,  0, 10,  4,  9,  5,  2,  1])

In [32]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear')
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8603174603174604


# Wrapper Method
sklearn v.ไม่ถึง ต้องอัพเป็น 0.2.4 + ยังไม่มีfunctionในสื่อการสอน รอก่อน

# Embedded Method
Ex. Random forest

----------------

In [33]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', LogisticRegression(random_state=0, solver='liblinear'))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 LogisticRegression(random_state=0, solver='liblinear'))])

In [34]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.765079365079365


# Assignment

# Feature Reduction + Feature Selection + Imbalance + Pipeline 

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0) 

In [36]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import f_classif  
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {
    'C' : [0.1,1,10],
    'gamma' : [0.0001,0.002,0.04,0.8,1.8,3.2,5,10],
    'break_ties' : ['True']
}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('decomposition', PCA(n_components = 5)),
  ('scaler', MinMaxScaler(feature_range=(0, 1))), 
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(SVC(random_state=0),params, cv = 10))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('decomposition', PCA(n_components=5)),
                ('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 GridSearchCV(cv=10, estimator=SVC(random_state=0),
                              param_grid={'C': [0.1, 1, 10],
                                          'break_ties': ['True'],
                                          'gamma': [0.0001, 0.002, 0.04, 0.8,
                                                    1.8, 3.2, 5, 10]}))])

In [37]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()))

10CV Training Accuracy : 0.7390124342419895


In [38]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.6793650793650794


-------------------

# Feature Selection + Imbalance + Pipeline 

In [39]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import f_classif  
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {
    'C' : [0.1,1,10],
    'gamma' : [0.0001,0.002,0.04,0.8,1.8,3.2,5,10],
    'break_ties' : ['True']
}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))), 
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(SVC(random_state=0),params, cv = 10))
])
clf.fit(X_res, y_res)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('feature_selection', SelectKBest(k=5)),
                ('classification',
                 GridSearchCV(cv=10, estimator=SVC(random_state=0),
                              param_grid={'C': [0.1, 1, 10],
                                          'break_ties': ['True'],
                                          'gamma': [0.0001, 0.002, 0.04, 0.8,
                                                    1.8, 3.2, 5, 10]}))])

In [40]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(clf, X_res, y_res, cv=10)
print("10CV Training Accuracy : "+str(acc.mean()))

10CV Training Accuracy : 0.7860090865614537


In [41]:
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test accuracy : "+str(acc))

Test accuracy : 0.8666666666666667
